# Train and hyperparameter tune with Scikit-learn

## Prerequisites

- Create an Azure ML Workspace and setup environmnet on local computer following the steps in [Azure README.md](https://gitlab-master.nvidia.com/drobison/aws-sagemaker-gtc-2020/tree/master/azure/README.md )

In [ ]:
# verify installation and check Azure ML SDK version
import azureml.core

print('SDK version:', azureml.core.VERSION)

## Create a FileDataset

In this example, we will use 20 million rows (samples) of the [airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html). The [FileDataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.filedataset?view=azure-ml-py) below references parquet files that have been uploaded to a public [Azure Blob storage](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-blobs-overview), you can download to your local computer or mount the files to your AML compute.

In [ ]:
from azureml.core.dataset import Dataset
airline_ds = Dataset.File.from_files('https://airlinedataset.blob.core.windows.net/airline-20m/*')

# larger dataset (10 years of airline data) is also available for multi-GPU option
# airline_ds = Dataset.File.from_files('https://airlinedataset.blob.core.windows.net/airline-10years/*')

In [ ]:
# download the dataset as local files
airline_ds.download(target_path='/local/path')

## Initialize workspace

Load and initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`

In [ ]:
from azureml.core.workspace import Workspace

# if a locally-saved configuration file for the workspace is not available, use the following to load workspace
# ws = Workspace(subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name)

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

datastore = ws.get_default_datastore()
print("Default datastore's name: {}".format(datastore.name))

## Upload data

Upload the dataset to the workspace's default datastore:

In [ ]:
path_on_datastore = 'data_airline'
datastore.upload(src_dir='/add/local/path', target_path=path_on_datastore, overwrite=False, show_progress=True)

In [ ]:
ds_data = datastore.path(path_on_datastore)
print(ds_data)

## Create AML compute

You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this notebook, we will use Azure ML managed compute ([AmlCompute](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)) for our remote training using a dynamically scalable pool of compute resources.

This notebook will use 10 nodes for hyperparameter optimization, you can modify `max_node` based on available quota in the desired region. Similar to other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. [This article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) includes details on the default limits and how to request more quota.

If we could not find the cluster with the given name, then we will create a new cluster here.`vm_size` describes the virtual machine type and size that will be used in the cluster. You will need to specify compute targets from [virtual machines in Azure](https://docs.microsoft.com/en-us/azure/virtual-machines/linux/sizes). Let's create an `AmlCompute` cluster of `Standard_DS5_v2` VMs:

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cpu_cluster_name = "cpu-cluster"

if cpu_cluster_name in ws.compute_targets:
    cpu_cluster = ws.compute_targets[cpu_cluster_name]
    if cpu_cluster and type(cpu_cluster) is AmlCompute:
        print('Found compute target. Will use {0} '.format(cpu_cluster_name))
else:
    print("creating new cluster")

    provisioning_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_DS5_v2', max_nodes = 10, idle_seconds_before_scaledown = 300)

    # create the cluster
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    cpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
# use get_status() to get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

## Prepare training script

Create a project directory that will contain code from your local machine that you will need access to on the remote resource. This includes the training script and additional files your training script depends on. In this example, the training script is provided `train_sklearn_RF.py`.

In [ ]:
import os

project_folder = './train_sklearn'
os.makedirs(project_folder, exist_ok=True)

We will log some metrics by using the `Run` object within the training script:

```python
from azureml.core.run import Run
run = Run.get_context()
```
 
We will also log the parameters and highest accuracy the model achieves:

```python
run.log('Accuracy', np.float(accuracy))
```


These run metrics will become particularly important when we begin hyperparameter tuning our model in the 'Tune model hyperparameters' section.

Once your script is ready, copy the training script `train_sklearn_RF.py` into your project directory:

In [ ]:
import shutil

shutil.copy('../code/train_sklearn_RF.py', project_folder)

## Train model on the remote compute

Now that you have your data and training script prepared, you are ready to train on your remote compute.

### Create an experiment

Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace.

In [ ]:
from azureml.core import Experiment

experiment_name = 'train_sklearn'
experiment = Experiment(ws, name=experiment_name)

### Create a scikit-learn Estimator

In [ ]:
from azureml.train.sklearn import SKLearn

script_params = {
    '--data_dir': ds_data.as_mount(),
}

estimator = SKLearn(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=cpu_cluster,
                    entry_script='train_sklearn_RF.py',
                    pip_packages=['pyarrow'])

## Tune model hyperparameters

We can optimize our model's hyperparameters and improve the accuracy using Azure Machine Learning's hyperparameter tuning capabilities.

### Start a hyperparameter sweep

Let's define the hyperparameter space to sweep over. We will tune `n_estimators`, `max_depth` and `max_features` parameters. In this example we will use random sampling to try different configuration sets of hyperparameters and maximize `Accuracy`.

In [ ]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice, loguniform, uniform

param_sampling = RandomParameterSampling( {
    '--n_estimators': choice(range(50, 500)),
    '--max_depth': choice(range(5, 19)),
    '--max_features': uniform(0.2, 1.0)
    }
)   

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                         hyperparameter_sampling=param_sampling, 
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=100,
                                         max_concurrent_runs=10)

This will launch the training script with parameters that were specified in the cell above.

In [ ]:
# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Monitor HyperDrive runs

Monitor and view the progress of the machine learning training run with a [Jupyter widget](https://docs.microsoft.com/en-us/python/api/azureml-widgets/azureml.widgets?view=azure-ml-py).The widget is asynchronous and provides live updates every 10-15 seconds until the job completes.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(hyperdrive_run).show()

In [ ]:
# hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
# hyperdrive_run.cancel()

### Find and register best model

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

List the model files uploaded during the run:

In [ ]:
print(best_run.get_file_names())

Register the folder (and all files in it) as a model named `train-sklearn` under the workspace for deployment

In [ ]:
# model = best_run.register_model(model_name='train-sklearn', model_path='outputs/model-sklearn.joblib')

## Delete cluster

In [ ]:
# delete the cluster
# gpu_cluster.delete()